In [2]:
import pandas as pd
import re
from typing import Any, Literal
from dataclasses import dataclass, asdict,field
import csv
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np


In [3]:
#Tutorial Base
#Classificadr multi label 

#https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html

In [4]:
df = pd.read_excel("C:\classificacaomultilabel\moveis.xlsx")
df.head(4)

,IDVENDA,fabricanteId,observacoes,dataCadastro,statusId,totalPedido,valorFinalPedido,descritivoPlanejados,valorFrete,pontoAtencao,NomeFabricante,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,41,11,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,2022-08-08 14:09:51.493,10.0,0.0,NaN,[Ambiente: cozinha - Obs: lacca branca somente...,NaN,NaN,HR MÓVEIS - HRM INDUSTRIA DE MOVEIS EM MDF E ...,NaN,NaN,NaN,NaN
1,41,12,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,2022-08-08 14:09:51.493,10.0,0.0,NaN,[Ambiente: cozinha - Obs: lacca branca somente...,NaN,NaN,EURONOBRE VIDROS LTDA,NaN,NaN,NaN,NaN
2,41,14,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,2022-08-08 14:09:51.493,10.0,0.0,NaN,[Ambiente: cozinha - Obs: lacca branca somente...,NaN,NaN,MYBOX ESPECIAIS - M D COMERCIO E MONTAGEM DE M...,NaN,NaN,NaN,NaN
3,41,15,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,2022-08-08 14:09:51.493,10.0,0.0,NaN,[Ambiente: cozinha - Obs: lacca branca somente...,NaN,NaN,HR LACA - HRM INDUSTRIA DE MOVEIS EM MDF,NaN,NaN,NaN,NaN


In [5]:
def case_function(valores):
        
        if re.findall('([Cozinha]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Cozinha"

        elif re.findall('([Quarto]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Quarto"

        elif re.findall('([Rack]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Sala"
        
        elif re.findall('([closet]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Closet"

        elif re.findall('([BANHO]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Banheiro"

        elif re.findall('([MESA GAMER]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Escritório"

        elif re.findall('([Escritóorio]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Escritório"

        elif re.findall('([Sanitáario]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Banheiro"

        elif re.findall('([Sala]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Sala"

        elif re.findall('([Dormitorio ]\-)+',str(valores), re.IGNORECASE):
            yield "Quarto"

        elif re.findall('([Aparador]\s+\-)+',str(valores), re.IGNORECASE):
            yield "Sala de Estar"

        elif re.findall('([Quarto]\s+[A-Z]\w+\s\-)+',str(valores), re.IGNORECASE):
            yield "Quarto"

        elif re.findall('([Dormitóorio]\s+[A-Z]\w+\s\-)+',str(valores), re.IGNORECASE):
            yield "Quarto"

        
        elif re.findall('([Sala de Estar]\s+[A-Z]\w+\s\-)+',str(valores), re.IGNORECASE):
            yield "Sala de Estar"
        
        else:
            
            yield "Não classificado"

In [6]:
class ReaderFiles:
    file: str = r"C:\classificacaomultilabel\moveis.xlsx"   


    def csv_reader(self):
        data = pd.read_excel(r"C:\classificacaomultilabel\moveis.xlsx" )
        print(data.columns)
        rows = data.to_dict("records")
      
        yield rows
              
    def classifica(self):
        lista_dicts = []
        rows = self.csv_reader()
        iterows = [{**row} for row in chain.from_iterable(rows)]
        
        cont = len(iterows)
        i = 0
        while i < cont:
            nam = str(iterows[i]['descritivoPlanejados']).strip().replace("/"," ").replace(
                "[","").replace("]","").replace("-","").replace("Ambiente:","").replace(
                "Obs: ","").replace("  "," ").replace("Ó ","O ").replace("Á","A").capitalize()
 
            items = case_function(iterows[i]['descritivoPlanejados'])
            for item in items:
                
                dict_ambiente = {
                    "Departamento":item,
                    "statusId":iterows[i]["statusId"],
                    "totalPedido":iterows[i]["totalPedido"],
                    "valorFinalPedido":iterows[i]["valorFinalPedido"],
                    "descritivoPlanejados":nam,
                    "observacoes":iterows[i]["observacoes"],
                    "IDVENDA":iterows[i]["IDVENDA"],
                    "dataCadastro":iterows[i]["dataCadastro"],
                    "fabricanteId":iterows[i]["fabricanteId"],
                    "NomeFabricante":iterows[i]["NomeFabricante"]}

                lista_dicts.append(dict_ambiente)
            i +=1


        return lista_dicts
reader = ReaderFiles()
lista_df = reader.classifica()
dados = pd.DataFrame(lista_df)


dados = dados[dados['Departamento'] !="Não classificado"]


Index(['IDVENDA', 'fabricanteId', 'observacoes', 'dataCadastro', 'statusId',
       'totalPedido', 'valorFinalPedido', 'descritivoPlanejados', 'valorFrete',
       'pontoAtencao', 'NomeFabricante', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14'],
      dtype='object')


In [7]:
names = dados.Departamento.unique()

In [8]:

lista_de_tags = list()

for nam in names:
    if nam not in lista_de_tags:
        lista_de_tags.append(nam)

print(lista_de_tags)



['Cozinha', 'Escritório', 'Quarto', 'Closet', 'Banheiro', 'Sala']


In [9]:

def add_new_column(lista_tags, dataframe, nome_tags):
    for tag in lista_tags:
        coluna = list()
        for lines in dataframe[nome_tags]:
            if tag in lines:
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna

add_new_column(lista_de_tags, dados, "Departamento")


In [10]:
dados.head(2)

,Departamento,statusId,totalPedido,valorFinalPedido,descritivoPlanejados,observacoes,IDVENDA,dataCadastro,fabricanteId,NomeFabricante,Cozinha,Escritório,Quarto,Closet,Banheiro,Sala
0,Cozinha,10.0,0.0,NaN,cozinha lacca branca somente nas portas gourm...,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,41,2022-08-08 14:09:51.493,11,HR MÓVEIS - HRM INDUSTRIA DE MOVEIS EM MDF E ...,1,0,0,0,0,0
1,Cozinha,10.0,0.0,NaN,cozinha lacca branca somente nas portas gourm...,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,41,2022-08-08 14:09:51.493,12,EURONOBRE VIDROS LTDA,1,0,0,0,0,0


In [11]:
lista_de_tags

['Cozinha', 'Escritório', 'Quarto', 'Closet', 'Banheiro', 'Sala']

In [13]:
cont = len(lista_de_tags)
i = 0

    
zip_items = list(zip(dados[lista_de_tags[0]],
               dados[lista_de_tags[1]],
               dados[lista_de_tags[2]],
               dados[lista_de_tags[3]],
               dados[lista_de_tags[4]]))
  

dados["all_departamentos"] = zip_items

dados.head(2)

,Departamento,statusId,totalPedido,valorFinalPedido,descritivoPlanejados,observacoes,IDVENDA,dataCadastro,fabricanteId,NomeFabricante,Cozinha,Escritório,Quarto,Closet,Banheiro,Sala,all_departamentos
0,Cozinha,10.0,0.0,NaN,cozinha lacca branca somente nas portas gourm...,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,41,2022-08-08 14:09:51.493,11,HR MÓVEIS - HRM INDUSTRIA DE MOVEIS EM MDF E ...,1,0,0,0,0,0,"(1, 0, 0, 0, 0)"
1,Cozinha,10.0,0.0,NaN,cozinha lacca branca somente nas portas gourm...,[PLANEJADOS] Pedido Criado em 08/08/2022 14:09...,41,2022-08-08 14:09:51.493,12,EURONOBRE VIDROS LTDA,1,0,0,0,0,0,"(1, 0, 0, 0, 0)"


In [594]:
"""Separando dados de teste e treino"""

'Separando dados de teste e treino'

In [22]:
X_treino, X_test, Y_treino, Y_test = train_test_split(
    dados.descritivoPlanejados,
    dados.all_departamentos,
    test_size = 0.2,
    random_state=123
)


print(Y_treino,X_test,Y_treino,Y_test)


1562     (1, 0, 0, 0, 0)
7800     (0, 1, 0, 0, 0)
13162    (1, 0, 0, 0, 0)
10919    (1, 0, 0, 0, 0)
8289     (1, 0, 0, 0, 0)
              ...       
5236     (1, 0, 0, 0, 0)
12539    (1, 0, 0, 0, 0)
1364     (0, 1, 0, 0, 0)
11910    (1, 0, 0, 0, 0)
3600     (1, 0, 0, 0, 0)
Name: all_departamentos, Length: 12068, dtype: object 9401      escritório dois nichos menores em preto trama...
12801     mb 01 porta vaivem porta em mdf 36mm na cor b...
9799      cozinha uma cozinha composta por um armário a...
5621      painel de tv e adega  painéis em mdf 18mm: bo...
8895      balcão de cozinha balcão com uma gaveta e dua...
                               ...                        
12603     tampo mesa redondo 90x90 tampo 90x90 redondo ...
14457     cozinha cozinha, caixarias em 15mm padrão, br...
10837     cozinha  caixaria em mdf de 15mm no padrão br...
5916                                              cozinha 
2868      banheiro painéis em mdf branco 15mm, caixaria...
Name: descritivoPlanej

In [24]:
vetor = TfidfVectorizer(max_features=50000, max_df=0.85)
vetor.fit(dados.descritivoPlanejados)

TfidfVectorizer(max_df=0.85, max_features=50000)

In [ ]:
X_treino_tfid =vetor.transform(X_treino)
X_test_tfid = vetor.transform(Y_test)

In [ ]:
print(X_treino_tfid.shape)

(12068, 3836)


In [ ]:

regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest.fit(X_treino_tfid, X_test_tfid)

In [ ]:
tags_treino_array = np.asarray(list(X_treino))
tags_test_array = np.asarray(list(X_test))

In [ ]:
X_treino_tfid =vetor.transform(X_treino)
X_test_tfid = vetor.transform(X_test)

In [ ]:
"""Regressão logistica"""


'Regressão logistica'

In [ ]:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest.fit(X_treino_tfid, X_test_tfid)